In [19]:
root_url='https://search-proquest-com.lib-ezproxy.hkbu.edu.hk/asianews/publication/26858/citation/6F63087B31B643A0PQ/12?accountid=11440'
from selenium import webdriver
import time 
browser = webdriver.Chrome() #initiate webdriver
browser.get(root_url)
pages=[
    ["filterdecade2010-2019","filteryear20152010-2018","filtermonth022015","filterissueNameFeb10,2015February"],
    ["filterdecade2010-2019","filteryear20152010-2018","filtermonth022015","filterissueNameFeb11,2015February"],
    ["filterdecade2010-2019","filteryear20152010-2018","filtermonth022015","filterissueNameFeb14,2015February"],
    ["filterdecade2010-2019","filteryear20152010-2018","filtermonth022015","filterissueNameFeb22,2015February"],
    ["filterdecade2010-2019","filteryear20152010-2018","filtermonth022015","filterissueNameFeb24,2015February"],
    ["filterdecade2010-2019","filteryear20152010-2018","filtermonth032015","filterissueNameMar20,2015March"],
]

In [20]:
def scrape_one_page():
    articles=browser.find_elements_by_class_name("resultItem")
    data=[]
    for article in articles:
        headline=article.find_element_by_class_name("truncatedResultsTitle").text
        byline=article.find_elements_by_class_name("titleAuthorETC")[0].text
        byline2=article.find_elements_by_class_name("titleAuthorETC")[1].text
        data.append((headline,byline,byline2,datestr))
    return data 

In [21]:
SLEEP=5
all_data=[]
for page in pages:
    try:
        browser.get(root_url)
        time.sleep(SLEEP)
        for element in page:
            browser.find_element_by_id(element).click()
            time.sleep(SLEEP)
        datestr=page[-1]
        data=scrape_one_page()
        all_data.extend(data)
        nextpage=browser.find_element_by_css_selector("#lor_container > div.pub_bottom_sort_by > nav > ul > li:nth-child(5) > a")
        nextpage.click() 
        datestr=page[-1]
        data=scrape_one_page()
        all_data.extend(data)
    except Exception as e:
        print("Failed on",page)
        print(e)

Failed on ['filterdecade2010-2019', 'filteryear20152010-2018', 'filtermonth22015', 'filterissueNameFeb10,2015February']
Message: no such element: Unable to locate element: {"method":"id","selector":"filtermonth22015"}
  (Session info: chrome=69.0.3497.92)
  (Driver info: chromedriver=2.43.600210 (68dcf5eebde37173d4027fa8635e332711d2874a),platform=Windows NT 10.0.17134 x86_64)

Failed on ['filterdecade2010-2019', 'filteryear20152010-2018', 'filtermonth022015', 'filterissueNameFeb11,2015February']
Message: no such element: Unable to locate element: {"method":"id","selector":"filterissueNameFeb11,2015February"}
  (Session info: chrome=69.0.3497.92)
  (Driver info: chromedriver=2.43.600210 (68dcf5eebde37173d4027fa8635e332711d2874a),platform=Windows NT 10.0.17134 x86_64)

Failed on ['filterdecade2010-2019', 'filteryear20152010-2018', 'filtermonth022015', 'filterissueNameFeb22,2015February']
Message: no such element: Unable to locate element: {"method":"css selector","selector":"#lor_contain

In [22]:
import pandas as pd 
df=pd.DataFrame(all_data,columns=["title","byline","byline2","datestr"])
df
for i in range(len(df["byline2"])):
    df['byline2'][i] = df['byline2'][i].split(']')[-1]
df["byline2"]
for i in range(len(df["byline2"])):
    df["byline2"][i]=df["byline2"][i].split(":")[0]
df["byline2"]
df
del df["datestr"]
df.columns=["title","byline","dates"]
df.to_csv('scmpxw2015b.csv', encoding='utf-8')